BART는 사전학습된 시퀀스-투-시퀀스 모델로서 노이즈를 제거하는 오토인코더 입니다.

여기서 시퀀스 투 시퀀스 모델이란 입력 시퀀스(원소들이 한 줄로 길게 늘어선 모양)로 넣고,

출력도 시퀀스로 받는 모델을 의미합니다.
BART는 임의의 노이즈 함수로 텍스트에 변동성을 추가하고 난 후 원래 텍스트를 재구축하도록 모델을 학습을 진행함.

BART는 신경망 번역에 사용되는 통상적인 트랜스포머를 씁니다. BART는 신경망 번역에 사용되는 통상적인 트랜스포머를 씁니다.

이 모델은 구조가 단순하다는 장점과 함께, BERT나 GPT 등 최신 사전학습 모델의 일반화 형태로 간주되기도 합니다.

왜냐하면 BART는 인코더로 BERT(양방향 인코더로 구성)를 사용하고, 디코더 GPT(단방향 디코더로 구성)를 사용하기 떄문입니다.

이 장에서는 BART를 사용하여 문장 자동 요약을 시도합니다.

인코딩 <-> 디코딩 쌍방 변환.
input network 수 = output netowrk 수

원본 데이터에 있는 정보 중에서 핵심이 되는 정보를 파악하는데 방해가 되는 잡음 정보.
디 노이징.


In [1]:
# facebook/bart-large 모델 및 토크나이저
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large')
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [2]:
import re

article = """
Google is an American multinational corporation and technology company focusing on online advertising, search engine technology, cloud computing, computer software, quantum computing, e-commerce, consumer electronics, and artificial intelligence (AI).[9] It has been referred to as "the most powerful company in the world" by the BBC[10] and is one of the world's most valuable brands due to its market dominance, data collection, and technological advantages in the field of AI.[11][12][13] Alongside Amazon, Apple, Meta, and Microsoft, Google's parent company, Alphabet Inc. is one of the five Big Tech companies.

Google was founded on September 4, 1998, by American computer scientists Larry Page and Sergey Brin while they were PhD students at Stanford University in California. Together, they own about 14% of its publicly listed shares and control 56% of its stockholder voting power through super-voting stock. The company went public via an initial public offering (IPO) in 2004. In 2015, Google was reorganized as a wholly owned subsidiary of Alphabet Inc. Google is Alphabet's largest subsidiary and is a holding company for Alphabet's internet properties and interests. Sundar Pichai was appointed CEO of Google on October 24, 2015, replacing Larry Page, who became the CEO of Alphabet. On December 3, 2019, Pichai also became the CEO of Alphabet.[14]

The company has since rapidly grown to offer a multitude of products and services beyond Google Search, many of which hold dominant market positions. These products address a wide range of use cases, including email (Gmail), navigation and mapping (Waze, Maps and Earth), cloud computing (Cloud), web navigation (Chrome), video sharing (YouTube), productivity (Workspace), operating systems (Android), cloud storage (Drive), language translation (Translate), photo storage (Photos), videotelephony (Meet), smart home (Nest), smartphones (Pixel), wearable technology (Pixel Watch and Fitbit), music streaming (YouTube Music), video on demand (YouTube TV), AI (Google Assistant and Gemini), machine learning APIs (TensorFlow), AI chips (TPU), and more. Discontinued Google products include gaming (Stadia),[15] Glass, Google+, Reader, Play Music, Nexus, Hangouts, and Inbox by Gmail.[16][17] Google's other ventures outside of internet services and consumer electronics include quantum computing (Sycamore), self-driving cars (Waymo, formerly the Google Self-Driving Car Project), smart cities (Sidewalk Labs), and transformer models (Google DeepMind).[18]

Google Search and YouTube are the two most-visited websites worldwide followed by Facebook and Twitter (now known as X). Google is also the largest search engine, mapping and navigation application, email provider, office suite, online video platform, photo and cloud storage provider, mobile operating system, web browser, machine learning framework, and AI virtual assistant provider in the world as measured by market share.[19] On the list of most valuable brands, Google is ranked second by Forbes as of January 2022[20] and fourth by Interbrand as of February 2022.[21]

The company has received significant criticism involving issues such as privacy concerns, tax avoidance, censorship, search neutrality, antitrust and abuse of its monopoly position. On August 5, 2024, D.C. Circuit Court Judge Amit P. Mehta ruled that Google held an illegal monopoly over Internet search.[22]"""

article = re.sub(r"[:.]\[[0-9]+\]\([0-9]+\)|.?[([][0-9]+[])]|\n|\r", r"", article)

print(article)

Google is an American multinational corporation and technology company focusing on online advertising, search engine technology, cloud computing, computer software, quantum computing, e-commerce, consumer electronics, and artificial intelligence (AI) It has been referred to as "the most powerful company in the world" by the BB and is one of the world's most valuable brands due to its market dominance, data collection, and technological advantages in the field of AI Alongside Amazon, Apple, Meta, and Microsoft, Google's parent company, Alphabet Inc. is one of the five Big Tech companies.Google was founded on September 4, 1998, by American computer scientists Larry Page and Sergey Brin while they were PhD students at Stanford University in California. Together, they own about 14% of its publicly listed shares and control 56% of its stockholder voting power through super-voting stock. The company went public via an initial public offering (IPO) in 2004. In 2015, Google was reorganized as 

In [13]:
# 모델 학습 이전에 BART 토크나이징
# 바트 모델을 사용해서 입력 텍스트를 토크나이징 하세요.
inputs = tokenizer.prepare_seq2seq_batch(article, return_tensors='pt', truncation=True)
inputs

{'input_ids': tensor([[    0, 20441,    16,    41,   470, 17043, 12772,     8,   806,   138,
          5650,    15,   804,  4579,     6,  1707,  3819,   806,     6,  3613,
         11730,     6,  3034,  2257,     6, 17997, 11730,     6,   364,    12,
          8342,     6,  2267,  8917,     6,     8,  7350,  2316,    36, 15238,
            43,    85,    34,    57,  4997,     7,    25,    22,   627,   144,
          2247,   138,    11,     5,   232,   113,    30,     5,  8112,     8,
            16,    65,     9,     5,   232,    18,   144,  5130,  3595,   528,
             7,    63,   210, 12532,     6,   414,  2783,     6,     8,  9874,
         12340,    11,     5,   882,     9,  4687,  7538,  3730,  1645,     6,
          1257,     6, 37622,     6,     8,  3709,     6,  1204,    18,  4095,
           138,     6, 15023,   603,     4,    16,    65,     9,     5,   292,
          1776,  4569,   451,     4, 20441,    21,  4790,    15,   772,   204,
             6,  6708,     6,    30,  

In [20]:
# bert 토크나이징 결과를 넘파이 어레이로 변환.
inputs['input_ids'].numpy()

array([[    0, 20441,    16,    41,   470, 17043, 12772,     8,   806,
          138,  5650,    15,   804,  4579,     6,  1707,  3819,   806,
            6,  3613, 11730,     6,  3034,  2257,     6, 17997, 11730,
            6,   364,    12,  8342,     6,  2267,  8917,     6,     8,
         7350,  2316,    36, 15238,    43,    85,    34,    57,  4997,
            7,    25,    22,   627,   144,  2247,   138,    11,     5,
          232,   113,    30,     5,  8112,     8,    16,    65,     9,
            5,   232,    18,   144,  5130,  3595,   528,     7,    63,
          210, 12532,     6,   414,  2783,     6,     8,  9874, 12340,
           11,     5,   882,     9,  4687,  7538,  3730,  1645,     6,
         1257,     6, 37622,     6,     8,  3709,     6,  1204,    18,
         4095,   138,     6, 15023,   603,     4,    16,    65,     9,
            5,   292,  1776,  4569,   451,     4, 20441,    21,  4790,
           15,   772,   204,     6,  6708,     6,    30,   470,  3034,
      

In [35]:
# 문제 052의 영문 위키피디아의 Google LCC 텍스트 처리 결과물을 문제 050에서 불러온 모델에 투입해서 요약 문장을
# vocabulary ID 형태로 생성하세요.
outputs = model.generate(inputs['input_ids'])
print(outputs)

tensor([[    2,     0, 20441,    16,    41,   470, 17043, 12772,     8,   806,
           138,  5650,    15,   804,  4579,     6,  1707,  3819,   806,     6,
             2]])


In [40]:
# BART 추론 결과를 디코딩.
# 문제 054에서 생성된 요약 문장을 summary_ids 을 디코딩해서 사람이 읽고 이해할 수 있는 자연스런 영어 문장으로 변환하세요^^

ds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]
print("".join(ds))

Google is an American multinational corporation and technology company focusing on online advertising, search engine technology,
